In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
file_reference = "purchase_data.csv"

In [3]:
sales_data = pd.read_csv(file_reference)

## Player Count

In [4]:
total_players = pd.DataFrame({"Total Players": [len(sales_data["SN"].unique())]}) 
total_players


,Total Players
0,576


## Purchasing Analysis (Total)




Run basic calculations to obtain number of unique items, average price, etc.
Create a summary data frame to hold the results
Optional: give the displayed data cleaner formatting
Display the summary data frame


In [5]:
unique_items = len(sales_data["Item ID"].unique())
purchases = sales_data["Price"].count()
total_revenue=sales_data["Price"].sum()
avg_price = total_revenue / purchases

summary_table = pd.DataFrame({"Number of Unique Items": [unique_items], 
                              "Average Price": [avg_price], 
                              "Number of Purchases": [purchases], 
                              "Total Revenue": [total_revenue]}) 
summary_table["Average Price"] = summary_table["Average Price"].map("${:.2f}".format)
summary_table["Total Revenue"] = summary_table["Total Revenue"].map("${:,.2f}".format) 
summary_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics




Percentage and Count of Male Players
Percentage and Count of Female Players
Percentage and Count of Other / Non-Disclosed


In [6]:
total_players = len(sales_data["SN"].unique()) 
grouped_players=sales_data.groupby('Gender')['SN'].nunique() 
n_sales_data = grouped_players.to_frame('Total Count') 
n_sales_data['Percentage of Players'] = round(n_sales_data/total_players*100,2) 
n_sales_data


,Total Count,Percentage of Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)
Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
Create a summary data frame to hold the results
Optional: give the displayed data cleaner formatting
Display the summary data frame


In [7]:
p_count = sales_data.groupby('Gender')['Item ID'].count()
p_value = sales_data.groupby('Gender')['Price'].sum()
purchase_analysis_df=p_count.to_frame('Purchase Count')
purchase_analysis_df["Average Purchase Price"] = (p_value/p_count).map("${:.2f}".format)
purchase_analysis_df["Total Purchase Value"] = p_value.map("${:.2f}".format)
purchase_analysis_df["Avg Total Purchase per Person"] = (p_value/grouped_players).map("${:.2f}".format)
purchase_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [8]:
age_bins = ([0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999])
group_names = (["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])
percent_of_players=pd.DataFrame(round((((pd.cut(sales_data["Age"], age_bins, labels=group_names)).value_counts())/total_players*100),2))
percent_of_players.rename(columns={'Age': 'Percentage of Players'},inplace = True)
percent_of_players["Total Count"]=pd.cut(sales_data["Age"], age_bins, labels=group_names).value_counts()
percent_of_players=percent_of_players.reindex(group_names)
percent_of_players

,Percentage of Players,Total Count
<10,3.99,23
10-14,4.86,28
15-19,23.61,136
20-24,63.37,365
25-29,17.53,101
30-34,12.67,73
35-39,7.12,41
40+,2.26,13


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [18]:
purchase_data = sales_data.copy()
purchase_data['Age']=pd.cut(sales_data['Age'], age_bins, labels=group_names)
by_age = purchase_data[['Price', 'Age']].groupby(['Age']).count()
by_age.rename(columns={'Price': 'Purchase Count'},inplace = True)
by_age["Average Purchase Price"] = purchase_data[['Price', 'Age']].groupby(['Age']).mean()
by_age['Total Purchase Price'] =purchase_data[['Price', 'Age']].groupby(['Age']).sum()
by_age['Normalized Totals'] = (by_age['Total Purchase Price'] / percent_of_players['Total Count'])
by_age['Average Purchase Price'] = by_age['Average Purchase Price'].map('${:,.2f}'.format)
by_age['Total Purchase Price'] = by_age['Total Purchase Price'].map('${:,.2f}'.format)
by_age['Normalized Totals'] = by_age['Normalized Totals'].map('${:,.2f}'.format)

by_age

,Purchase Count,Average Purchase Price,Total Purchase Price,Normalized Totals
Age,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,"$1,114.06",$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [19]:
top_spenders_df = sales_data['Item ID'].groupby(sales_data['SN']).count()
top_spenders_df = pd.DataFrame(data=top_spenders_df)
top_spenders_df.columns = ['Purchase Count']
top_spenders_df['Average Purchase Price'] = round(sales_data['Price'].groupby(sales_data['SN']).mean(),2)
top_spenders_df['Total Purchase Value'] = sales_data['Price'].groupby(sales_data['SN']).sum()
top_spenders_df.sort_values(by=['Total Purchase Value'], ascending=False, inplace=True)
top_spenders_df['Average Purchase Price'] = top_spenders_df['Average Purchase Price'].map('${:,.2f}'.format)
top_spenders_df['Total Purchase Value'] = top_spenders_df['Total Purchase Value'].map('${:,.2f}'.format)
top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [20]:
most_items_df = sales_data.groupby(['Item ID', 'Item Name','Price'])['Price'].agg(['count','sum'])
most_items_df.columns = ['Purchase Count', 'Total Purchase Value']
most_items_df.reset_index(inplace=True)
most_items_df.set_index(['Item ID','Item Name'] ,inplace=True)
most_items_df = most_items_df[['Purchase Count', 'Price', 'Total Purchase Value']]
most_pop_items_df = most_items_df.sort_values(by='Purchase Count', ascending=False)
most_pop_items_df['Price'] = most_pop_items_df['Price'].map('${:,.2f}'.format)
most_pop_items_df['Total Purchase Value'] = most_pop_items_df['Total Purchase Value'].map('${:,.2f}'.format)

most_pop_items_df.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [21]:
most_profit_items_df = most_items_df.sort_values(by='Total Purchase Value', ascending=False)
most_profit_items_df['Price'] = most_profit_items_df['Price'].map('${:,.2f}'.format)
most_profit_items_df['Total Purchase Value'] = most_profit_items_df['Total Purchase Value'].map('${:,.2f}'.format)
most_profit_items_df.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
